In [ ]:
import os
import sys
import threading
from PyQt5.QtWidgets import QApplication, QSystemTrayIcon, QMenu, QAction
from PyQt5.QtGui import QIcon
import pyscreenshot as ImageGrab
import requests
from bs4 import BeautifulSoup
from google.cloud import vision
import io
from tkinter import Tk, Label, Listbox, END

# Google Cloud Vision API credentials setup
# Make sure you have GOOGLE_APPLICATION_CREDENTIALS environment variable set to your service account key JSON file path

# Function to capture the screen
def capture_screen():
    print("Starting screen capture...")
    image = ImageGrab.grab()
    image.save("screenshot.png")
    print("Screen captured!")
    process_image("screenshot.png")

# Function to process the captured image
def process_image(image_path):
    print("Starting image processing...")
    search_image(image_path)

# Function to recognize image using Google Vision API
def recognize_image(image_path):
    print("Sending image to Google Vision API...")
    
    # Set the path to your service account key file
    credentials_path = os.path.join(os.path.dirname(__name__), 'my-project-1-441305-a75632af7501.json')
    
    # Create a Vision API client with explicit credentials
    client = vision.ImageAnnotatorClient.from_service_account_file(credentials_path)
    
    with io.open(image_path, 'rb') as image_file:
        content = image_file.read()
    image = vision.Image(content=content)
    response = client.label_detection(image=image)
    labels = response.label_annotations
    keywords = [label.description for label in labels]

    print("Received response from Google Vision API")
    return keywords

# Function to search for products on Flipkart
def search_flipkart(keywords):
    print("Searching on Flipkart...")
    query = '+'.join(keywords)
    url = f"https://www.flipkart.com/search?q={query}"
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        results = []
        for item in soup.find_all('div', {'class': '_1AtVbE'}):
            title = item.find('a', {'class': 'IRpwTa'})
            if title:
                title = title.text
                link = "https://www.flipkart.com" + item.find('a', {'class': 'IRpwTa'})['href']
                results.append((title, link))
        print("Flipkart search completed")
    except requests.exceptions.Timeout:
        print("Timeout occurred while searching Flipkart.")
        results = []
    return results

# Function to search for products on Myntra
def search_myntra(keywords):
    print("Searching on Myntra...")
    query = '-'.join(keywords)
    url = f"https://www.myntra.com/{query}"
    try:
        response = requests.get(url, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        results = []
        for item in soup.find_all('li', {'class': 'product-base'}):
            title = item.find('h4', {'class': 'product-product'})
            if title:
                title = title.text
                link = item.find('a', {'class': 'product-base'})['href']
                results.append((title, "https://www.myntra.com" + link))
        print("Myntra search completed")
    except requests.exceptions.Timeout:
        print("Timeout occurred while searching Myntra.")
        results = []
    return results

# Function to search for products on Amazon
def search_amazon(keywords):
    print("Searching on Amazon...")
    query = '+'.join(keywords)
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
    url = f"https://www.amazon.in/s?k={query}"
    try:
        response = requests.get(url, headers=headers, timeout=10)
        soup = BeautifulSoup(response.text, 'html.parser')
        results = []
        for item in soup.find_all('div', {'class': 's-main-slot'}):
            title = item.find('span', {'class': 'a-size-medium'})
            if title:
                title = title.text
                link = "https://www.amazon.in" + item.find('a', {'class': 'a-link-normal'})['href']
                results.append((title, link))
        print("Amazon search completed")
    except requests.exceptions.Timeout:
        print("Timeout occurred while searching Amazon.")
        results = []
    return results

# Function to display search results
def display_results(results):
    root = Tk()
    root.title("Search Results")
    
    for source, items in results.items():
        label = Label(root, text=source)
        label.pack()
        listbox = Listbox(root)
        for title, link in items:
            listbox.insert(END, f"{title} - {link}")
        listbox.pack()
    
    root.mainloop()

# Function to search image and display results
def search_image(image_path):
    keywords = recognize_image(image_path)
    
    flipkart_results = search_flipkart(keywords)
    myntra_results = search_myntra(keywords)
    amazon_results = search_amazon(keywords)

    results = {
        'Flipkart': flipkart_results,
        'Myntra': myntra_results,
        'Amazon': amazon_results
    }

    display_results(results)

# Main application setup using PyQt5 for system tray icon
def run_application():
    app = QApplication(sys.argv)
    icon = QSystemTrayIcon(QIcon('C:/Users/SAHIL/OneDrive/Desktop/PROJECTS/Final year Project\icon.png'), app)
    menu = QMenu()
    capture_action = QAction("Capture Screen")
    menu.addAction(capture_action)
    icon.setContextMenu(menu)
    icon.show()
    capture_action.triggered.connect(lambda: threading.Thread(target=capture_screen).start())  # Using threading for capture
    sys.exit(app.exec_())

if __name__ == "__main__":
    run_application()


Starting screen capture...
Screen captured!
Starting image processing...
Sending image to Google Vision API...
Received response from Google Vision API
Searching on Flipkart...
Flipkart search completed
Searching on Myntra...
Timeout occurred while searching Myntra.
Searching on Amazon...
Amazon search completed


In [ ]:
pip install PyQt5 pyscreenshot requests beautifulsoup4 google-cloud-vision google-auth
